## Language modelling

Objectives:

- Read the documentation of Language modelling in the Transformers library.

- Download three Polish models from the Huggingface repository. These should be regular language models, which were not fine-tuned. E.g. HerBERT and papuGaPT2 are good examples.

- Devise a method to test if the langage model understands Polish cases. E.g. testing for nominal case could be expressed as "Warszawa to największe [MASK]", and the masked word should be in nominative case. Create sentences for each case.

- Devise a method to test long-range relationships such as gender. E.e. you can use two verbs with masculine and feminine gender, where one of the verbs is masked. Both verbs should have the same gender, assuming the subject is the same. Define at least 3 such sentences.

- Check if the model captures real-world knolwedge. For instance a sentence "[MASK] wrze w temperaturze 100 stopni, a zamarza w temperaturze 0 stopni Celsjusza." checks if the model "knows" the description of water. Define at least 3 such sentences.

- Check zero-shot learning capabilites of the models. Provide at least 5 sentences with different sentiment for the following scheme: "'Ten film to był kiler. Nie mogłem się oderwać od ekranu.' Wypowiedź ta ma jest zdecydowanie [MASK]" Try different prompts, to see if they make any difference.

- Take into accout the fact, that causal language models such as PapuGaPT2 or plT5, will only generate continuations of the sentenes, so the examples have to be created according to that paradigm.

In [3]:
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoModel, AutoModelWithLMHead
import tensorflow as tf
import torch

## Download three Polish models from the Huggingface repository.

The pipeline function creates an instance of a pipeline for a specific NLP task. You provide the task name (e.g., "text-generation") and, optionally, the name or path of the model you want to use. The library then handles the details of loading the appropriate model and configuring it for the specified task.

### Loading papuGaPT2 model

In [4]:
# Load papuGA model directly
tokenizer_papuga = AutoTokenizer.from_pretrained("dkleczek/papuGaPT2")
model_papuga = AutoModelForCausalLM.from_pretrained("dkleczek/papuGaPT2")

### Loading HerBERT model

In [5]:
# Load model directly
tokenizer_herbert = AutoTokenizer.from_pretrained("allegro/herbert-base-cased")
model_herbert = AutoModel.from_pretrained("allegro/herbert-base-cased")

Some weights of the model checkpoint at allegro/herbert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.sso.sso_relationship.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.sso.sso_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


### Loading XLM-RoBERTa model

In [6]:

tokenizer_xlm = AutoTokenizer.from_pretrained("xlm-roberta-base")
model_xlm = AutoModel.from_pretrained("xlm-roberta-base")

## Devise a method to test if the langage model understands Polish cases

Example sentences in different cases for HerBERT and XLM models:

In [7]:
nominative = [
    "Pies to najlepszy <mask> człowieka.",
    "Widzę na niebie kolorową <mask>."
]

genitive = [
    "Jednak mój brat nie zbił <mask> w samochodzie.",
    "Ta ksiązka nalezy do <mask>."
]

dative = [
    "Sprzedałam mój rower młodszemu <mask>.",
    "Fajnie jest pomagać <mask> w potrzebie."
]

accusative = [
    "Widzę wysokie <mask> na horyzoncie.",
    "Uwielabim, gdy moja mama przygotowuje <mask>."
]

instrumental = [
    "Ostatnio częściej zdarza mi się pisać kolorowym <mask>.",
    "Najchętniej jezdzę do szkoły <mask>."
]

locative = [
    "Nie mogę przestać marzyć o <mask>.",
    "Bardzo długo toczyły się rozmowy o <mask>."
]

vocative = [
    "Ej <mask>, o której wrócisz?",
    "Bądź ostrozny na drodze, <mask>."
]

cases = [nominative, genitive, dative, accusative, instrumental, locative, vocative]

In [8]:
xlm = "xlm-roberta-base"
herbert = "allegro/herbert-base-cased"
papuga = "dkleczek/papuGaPT2"

In [9]:
def test_cases1(model, sentences):
    # Wybierzmy jeden model, na przykład xlm-roberta-base
    model_name = model

    # Pipeline do predykcji maski
    unmasker = pipeline('fill-mask', model=model_name)

    # Test predykcji dla jednego modelu
    print(f"Testing {model_name}:\n")
    for sentence in sentences:
        # Wygeneruj predykcje
        predictions = unmasker(sentence)
        
        completed_sentence = sentence.replace('<mask>', predictions[0]['token_str'])

        # Wyświetl wyniki
        print(f"Input Sentence: {sentence}")
        print(f"Generated Prediction: {predictions[0]['token_str']}")
        print(f"Predicted Sentence: {completed_sentence}\n")

Test XML RoBERTa:

In [10]:
for case in cases:
    test_cases1(xlm, case)
    print(40 * "_")

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForMaskedLM: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing XLMRobertaForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Testing xlm-roberta-base:

Input Sentence: Pies to najlepszy <mask> człowieka.
Generated Prediction: przyjaciel
Predicted Sentence: Pies to najlepszy przyjaciel człowieka.

Input Sentence: Widzę na niebie kolorową <mask>.
Generated Prediction: wodę
Predicted Sentence: Widzę na niebie kolorową wodę.

__________________________________


Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForMaskedLM: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing XLMRobertaForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Testing xlm-roberta-base:

Input Sentence: Jednak mój brat nie zbił <mask> w samochodzie.
Generated Prediction: się
Predicted Sentence: Jednak mój brat nie zbił się w samochodzie.

Input Sentence: Ta ksiązka nalezy do <mask>.
Generated Prediction: mnie
Predicted Sentence: Ta ksiązka nalezy do mnie.

__________________________________


Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForMaskedLM: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing XLMRobertaForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


...

Testing xlm-roberta-base:

Input Sentence: Ostatnio częściej zdarza mi się pisać kolorowym <mask>.
Generated Prediction: stylu
Predicted Sentence: Ostatnio częściej zdarza mi się pisać kolorowym stylu.

Input Sentence: Najchętniej jezdzę do szkoły <mask>.
Generated Prediction: .
Predicted Sentence: Najchętniej jezdzę do szkoły ..

__________________________________


Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForMaskedLM: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing XLMRobertaForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Testing xlm-roberta-base:

Input Sentence: Nie mogę przestać marzyć o <mask>.
Generated Prediction: życiu
Predicted Sentence: Nie mogę przestać marzyć o życiu.

Input Sentence: Bardzo długo toczyły się rozmowy o <mask>.
Generated Prediction: współpracy
Predicted Sentence: Bardzo długo toczyły się rozmowy o współpracy.

__________________________________


Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForMaskedLM: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing XLMRobertaForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Testing xlm-roberta-base:

Input Sentence: Ej <mask>, o której wrócisz?
Generated Prediction: da
Predicted Sentence: Ej da, o której wrócisz?

Input Sentence: Bądź ostrozny na drodze, <mask>.
Generated Prediction: a
Predicted Sentence: Bądź ostrozny na drodze, a.

__________________________________


Test HerBERT:

In [11]:
for case in cases:
    test_cases1(herbert, case)
    print(40 * "_")

Testing allegro/herbert-base-cased:

Input Sentence: Pies to najlepszy <mask> człowieka.
Generated Prediction: przyjaciel
Predicted Sentence: Pies to najlepszy przyjaciel człowieka.

Input Sentence: Widzę na niebie kolorową <mask>.
Generated Prediction: gwiazdę
Predicted Sentence: Widzę na niebie kolorową gwiazdę.

________________________________________
Testing allegro/herbert-base-cased:

Input Sentence: Jednak mój brat nie zbił <mask> w samochodzie.
Generated Prediction: szyby
Predicted Sentence: Jednak mój brat nie zbił szyby w samochodzie.

Input Sentence: Ta ksiązka nalezy do <mask>.
Generated Prediction: mnie
Predicted Sentence: Ta ksiązka nalezy do mnie.

________________________________________
Testing allegro/herbert-base-cased:

Input Sentence: Sprzedałam mój rower młodszemu <mask>.
Generated Prediction: koledze
Predicted Sentence: Sprzedałam mój rower młodszemu koledze.

Input Sentence: Fajnie jest pomagać <mask> w potrzebie.
Generated Prediction: ludziom
Predicted Sentenc

Example sentences in different cases for papuGaPT2:

In [12]:
nominative_papuga =  "Widzę na niebie kolorową"

genitive_papuga =  "Ta ksiązka nalezy do"

dative_papuga = "Sprzedał swój rower młodszemu"

accusative_papuga = "Uwielabim, gdy moja mama przygotowuje"

instrumental_papuga = "Właśnie pisze kolorowym"

locative_papuga = "Nie mogę przestać marzyć o"

vocative_papuga = "Bądź ostrozny na drodze,"

cases_papuga = [nominative_papuga, genitive_papuga, dative_papuga, accusative_papuga, instrumental_papuga, locative_papuga, vocative_papuga]

In [15]:
nominative_papuga2 =  "Dopiero skończyła się straszna burza. Przestało padać, więc wyszłam na zewnątrz. Widzę na niebie kolorową"

genitive_papuga2 =  "W tamtym tygodniu zrobił porządki w pokoju. Znalazł wiele ciekawych rzeczy, o których nie miał pojęcia. Ta ksiązka nalezy do"

dative_papuga2 = "Przez rok mój brat sporo urósł i jego rower juz na niego nie pasuje. Dlatego sprzedałał swój rower młodszemu"

accusative_papuga2 = "Jestem bardzo leniwa i nienawidzę robiś sobie jedzenia. Poza tym beznadziejnie gotuję. Uwielabim, gdy moja mama przygotowuje"

instrumental_papuga2 = "Niedawno załozył pamiętnik, w którym notuje swoje przezycia w ciągu dnia. Właśnie pisze kolorowym"

locative_papuga2 = "Mam dość szkoły. Ciągle tylko testy i kartkówki. Nie mogę przestać marzyć o"

vocative_papuga2 = "Wiem, ze wybeirasz się w dłuzszą trasę. Bądź ostrozny na drodze,"

cases_papuga2 = [nominative_papuga2, genitive_papuga2, dative_papuga2, accusative_papuga2, instrumental_papuga2, locative_papuga2, vocative_papuga2]

In [18]:
def test_cases_papuga(sentences):
    model = AutoModelWithLMHead.from_pretrained(papuga)
    tokenizer = AutoTokenizer.from_pretrained(papuga)

    for sentence in sentences:
        input_ids = tokenizer.encode(sentence, return_tensors='pt')
        attention_mask = torch.ones(input_ids.shape, device=input_ids.device)

        sample_outputs = model.generate(
            input_ids,
            attention_mask=attention_mask,
            do_sample=True,
            max_length=50,
            top_k=1,
            top_p=0.95,
            num_return_sequences=1,
            pad_token_id=tokenizer.eos_token_id
        )
        print("Output:" + 100 * '-')
        for i, sample_output in enumerate(sample_outputs):
            print("Input sentence: {} \nGenerated prediction: {}".format(sentence, tokenizer.decode(sample_output, skip_special_tokens=True)))


In [19]:
test_cases_papuga(cases_papuga)

Output:----------------------------------------------------------------------------------------------------
Input sentence: Widzę na niebie kolorową 
Generated prediction: Widzę na niebie kolorową gwiazdę, która świeci na niebie. W tym momencie, gdy słońce jest nisko, gwiazdy świecą na niebie.
W tym momencie, gdy słońce jest nisko, gwiazdy świecą na niebie. W tym momencie, gdy słońce jest nisko,
Output:----------------------------------------------------------------------------------------------------
Input sentence: Ta ksiązka nalezy do 
Generated prediction: Ta ksiązka nalezy do gatunku literatury popularnonaukowej, która w sposób przystępny i zrozumiały przedstawia zagadnienia związane z budową i funkcjonowaniem organizmu człowieka.
Książka jest przeznaczona dla osób, które chcą poznać budowę i funkcjonowanie organizmu człowieka, a także dla tych, którzy chcą
Output:----------------------------------------------------------------------------------------------------
Input sentence: S

In [20]:
test_cases_papuga(cases_papuga2)

Output:----------------------------------------------------------------------------------------------------
Input sentence: Dopiero skończyła się straszna burza. Przestało padać, więc wyszłam na zewnątrz. Widzę na niebie kolorową 
Generated prediction: Dopiero skończyła się straszna burza. Przestało padać, więc wyszłam na zewnątrz. Widzę na niebie kolorową chmurę. Jest to znak, że zbliża się burza.
Wstałam i poszłam do domu. W domu było już ciemno. W
Output:----------------------------------------------------------------------------------------------------
Input sentence: W tamtym tygodniu zrobił porządki w pokoju. Znalazł wiele ciekawych rzeczy, o których nie miał pojęcia. Ta ksiązka nalezy do 
Generated prediction: W tamtym tygodniu zrobił porządki w pokoju. Znalazł wiele ciekawych rzeczy, o których nie miał pojęcia. Ta ksiązka nalezy do jego ulubionych.
W tym tygodniu zrobił porządki w pokoju. Znalazł wiele ciekawych rzeczy, o których nie miał pojęcia. Ta
Output:--------------------

## Devise a method to test long-range relationships such as gender.

In [23]:
masculine = [
    "Szef wziął swój notes i <mask> na spotkanie z klientem.", "Mój ojciec zawsze <mask> naprawić zepsute sprzęty, ale tym razem nie dał rady,", "On przeczytał ksiązkę, po czym <mask>."]
feminine = ["Jego dziewczyna <mask> przez okno i stwierdziła, ze pora wyjść.", "Jakiś czas temu <mask> udział w ciekawym przedsięwzięciu, które potem pozwoliło jej otworzyć swój własny biznes.", "Ich córka najpierw bawiła się na placu zabaw, odpoczywała, a potem <mask>."]
neuter = ["Dziecko weszło do autobusu, <mask> kanapkę i wypiło sok.", "Nasze nowe auto <mask> w garazu.", "Koło najpierw się toczyło, a później <mask>."]


genders = [masculine, feminine, neuter]

In [24]:
for gender in genders:
    test_cases1(xlm, gender)

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForMaskedLM: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing XLMRobertaForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Testing xlm-roberta-base:

Input Sentence: Szef wziął swój notes i <mask> na spotkanie z klientem.
Generated Prediction: wpadł
Predicted Sentence: Szef wziął swój notes i wpadł na spotkanie z klientem.

Input Sentence: Mój ojciec zawsze <mask> naprawić zepsute sprzęty, ale tym razem nie dał rady,
Generated Prediction: próbował
Predicted Sentence: Mój ojciec zawsze próbował naprawić zepsute sprzęty, ale tym razem nie dał rady,

Input Sentence: On przeczytał ksiązkę, po czym <mask>.
Generated Prediction: napisał
Predicted Sentence: On przeczytał ksiązkę, po czym napisał.



Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForMaskedLM: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing XLMRobertaForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Testing xlm-roberta-base:

Input Sentence: Jego dziewczyna <mask> przez okno i stwierdziła, ze pora wyjść.
Generated Prediction: była
Predicted Sentence: Jego dziewczyna była przez okno i stwierdziła, ze pora wyjść.

Input Sentence: Jakiś czas temu <mask> udział w ciekawym przedsięwzięciu, które potem pozwoliło jej otworzyć swój własny biznes.
Generated Prediction: bierze
Predicted Sentence: Jakiś czas temu bierze udział w ciekawym przedsięwzięciu, które potem pozwoliło jej otworzyć swój własny biznes.

Input Sentence: Ich córka najpierw bawiła się na placu zabaw, odpoczytwała, a potem <mask>.
Generated Prediction: ...
Predicted Sentence: Ich córka najpierw bawiła się na placu zabaw, odpoczytwała, a potem ....



Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForMaskedLM: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing XLMRobertaForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Testing xlm-roberta-base:

Input Sentence: Dziecko weszło do autobusu, <mask> kanapkę i wypiło sok.
Generated Prediction: zostało
Predicted Sentence: Dziecko weszło do autobusu, zostało kanapkę i wypiło sok.

Input Sentence: Nasze nowe auto <mask> w garazu.
Generated Prediction: jest
Predicted Sentence: Nasze nowe auto jest w garazu.

Input Sentence: Koło najpierw się toczyło, a później <mask>.
Generated Prediction: ...
Predicted Sentence: Koło najpierw się toczyło, a później ....



In [25]:
for gender in genders:
    test_cases1(herbert, gender)

Testing allegro/herbert-base-cased:

Input Sentence: Szef wziął swój notes i <mask> na spotkanie z klientem.
Generated Prediction: poszedł
Predicted Sentence: Szef wziął swój notes i poszedł na spotkanie z klientem.

Input Sentence: Mój ojciec zawsze <mask> naprawić zepsute sprzęty, ale tym razem nie dał rady,
Generated Prediction: próbował
Predicted Sentence: Mój ojciec zawsze próbował naprawić zepsute sprzęty, ale tym razem nie dał rady,

Input Sentence: On przeczytał ksiązkę, po czym <mask>.
Generated Prediction: wyszedł
Predicted Sentence: On przeczytał ksiązkę, po czym wyszedł.

Testing allegro/herbert-base-cased:

Input Sentence: Jego dziewczyna <mask> przez okno i stwierdziła, ze pora wyjść.
Generated Prediction: wyszła
Predicted Sentence: Jego dziewczyna wyszła przez okno i stwierdziła, ze pora wyjść.

Input Sentence: Jakiś czas temu <mask> udział w ciekawym przedsięwzięciu, które potem pozwoliło jej otworzyć swój własny biznes.
Generated Prediction: wzięła
Predicted Sentence: 

In [105]:
masculine_papuga = "On przeczytał ksiązkę, po czym"
feminine_papuga = "Ich córka najpierw bawiła się na placu zabaw, odpoczytwała, a potem"
neuter_papuga = "Koło najpierw się toczyło, a później"


genders_papuga = [masculine_papuga, feminine_papuga, neuter_papuga]

In [106]:
test_cases_papuga(genders_papuga)

Output:----------------------------------------------------------------------------------------------------
Input sentence: On przeczytał ksiązkę, po czym 
Generated prediction: On przeczytał ksiązkę, po czym zaczął się śmiać.
- Nie, nie, nie.
Output:----------------------------------------------------------------------------------------------------
Input sentence: Ich córka najpierw bawiła się na placu zabaw, odpoczytwała, a potem 
Generated prediction: Ich córka najpierw bawiła się na placu zabaw, odpoczytwała, a potem poszła do szkoły.
Output:----------------------------------------------------------------------------------------------------
Input sentence: Koło najpierw się toczyło, a później 
Generated prediction: Koło najpierw się toczyło, a później się toczyło.
- Nie, nie, nie.


## Check if the model captures real-world knolwedge.

In [125]:
knowledge = [
    "Druga wojna światowa rozpoczęła się w <mask> roku.", #1939
    "<mask> jest stolicą Unii Europejskiej.", #Bruksela
    "Woda ma maksymalną gęstość przy temperaturze <mask> stopni Celsjusza.", #4
    "Góra o nazwie <mask> jest najwyższym szczytem w systemie górskim Himalajów." #Everest
]
knowledges = [knowledge]

In [126]:
for k in knowledges:
    test_cases1("allegro/herbert-base-cased", k)

Testing allegro/herbert-base-cased:

Input Sentence: Druga wojna światowa rozpoczęła się w <mask> roku.
Generated Prediction: 1939
Predicted Sentence: Druga wojna światowa rozpoczęła się w 1939 roku.

Input Sentence: <mask> jest stolicą Unii Europejskiej.
Generated Prediction: Bruksela
Predicted Sentence: Bruksela jest stolicą Unii Europejskiej.

Input Sentence: Woda ma maksymalną gęstość przy temperaturze <mask> stopni Celsjusza.
Generated Prediction: 30
Predicted Sentence: Woda ma maksymalną gęstość przy temperaturze 30 stopni Celsjusza.

Input Sentence: Góra o nazwie <mask> jest najwyższym szczytem w systemie górskim Himalajów.
Generated Prediction: Ta
Predicted Sentence: Góra o nazwie Ta jest najwyższym szczytem w systemie górskim Himalajów.



In [127]:
for k in knowledges:
    test_cases1("xlm-roberta-base", k)

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForMaskedLM: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing XLMRobertaForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Testing xlm-roberta-base:

Input Sentence: Druga wojna światowa rozpoczęła się w <mask> roku.
Generated Prediction: 1939
Predicted Sentence: Druga wojna światowa rozpoczęła się w 1939 roku.

Input Sentence: <mask> jest stolicą Unii Europejskiej.
Generated Prediction: Polska
Predicted Sentence: Polska jest stolicą Unii Europejskiej.

Input Sentence: Woda ma maksymalną gęstość przy temperaturze <mask> stopni Celsjusza.
Generated Prediction: 25
Predicted Sentence: Woda ma maksymalną gęstość przy temperaturze 25 stopni Celsjusza.

Input Sentence: Góra o nazwie <mask> jest najwyższym szczytem w systemie górskim Himalajów.
Generated Prediction: Everest
Predicted Sentence: Góra o nazwie Everest jest najwyższym szczytem w systemie górskim Himalajów.



In [130]:
knowledge_papuga = [
    "Druga wojna światowa rozpoczęła się w roku", #1939
    "Stolicą Unii Europejskiej jest", #Bruksela
    "Woda ma maksymalną gęstość w temperaturze o wartości", #4
    "Najwyzszym szczytem w systemie górskim Himalajów jest" #Everest
]

In [131]:
test_cases_papuga(knowledge_papuga)

Output:----------------------------------------------------------------------------------------------------
Input sentence: Druga wojna światowa rozpoczęła się w roku 
Generated prediction: Druga wojna światowa rozpoczęła się w roku 1914. W tym czasie w Europie i na świecie zapanowała epi
Output:----------------------------------------------------------------------------------------------------
Input sentence: Stolicą Unii Europejskiej jest 
Generated prediction: Stolicą Unii Europejskiej jest Luksemburg. W tym kraju, w którym w latach 90. XX
Output:----------------------------------------------------------------------------------------------------
Input sentence: Woda ma maksymalną gęstość w temperaturze o wartości 
Generated prediction: Woda ma maksymalną gęstość w temperaturze o wartości około 10°C. W przypadku, gdy temperatura otoczenia jest
Output:----------------------------------------------------------------------------------------------------
Input sentence: Najwyzszym szczyte

## Check zero-shot learning capabilites of the models.

In [57]:
sentiment_negative = ["'Jak ja go nie lubię.' Zdanie to ma <mask> wydźwięk", "'Pogoda dzisiaj jest beznadziejna.' Zdanie to ma wydźwięk <mask>", "'Ten film to był kiler. Nie mogłem się oderwać od ekranu.' Wypowiedź ta ma jest zdecydowanie <mask>"]
sentiment_positive = ["'To jest mój ulubiony serial!' Zdanie to ma wydźwięk <mask>.", "'Jak ładnie dzisiaj jest na dworze!' Zdanie to ma wydźwięk <mask>.", "'Zycie jest takie piękne!' Zdanie to ma wydźwięk <mask>."]

sentiments = [sentiment_negative, sentiment_positive]

sentiment_negative2 = ["'Jak ja go nie lubię.' Zdanie to jest nacechowane <mask>.", "'Pogoda dzisiaj jest beznadziejna.' Zdanie to jest nacechowane <mask>.", "'Nic mi się nie chce!' Zdanie to jest nacechowane <mask>."]
sentiment_positive2 = ["'To jest mój ulubiony serial!' Zdanie to jest nacechowane <mask>.", "'Jak ładnie dzisiaj jest na dworze!' Zdanie to jest nacechowane <mask>.", "'Zycie jest takie piękne!' Zdanie to jest nacechowane <mask>."]

sentiments2 = [sentiment_negative2, sentiment_positive2]

In [58]:
for s in sentiments:
    test_cases1(xlm, s)

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForMaskedLM: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing XLMRobertaForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Testing xlm-roberta-base:

Input Sentence: 'Jak ja go nie lubię.' Zdanie to ma <mask> wydźwięk
Generated Prediction: swój
Predicted Sentence: 'Jak ja go nie lubię.' Zdanie to ma swój wydźwięk

Input Sentence: 'Pogoda dzisiaj jest beznadziejna.' Zdanie to ma wydźwięk <mask>
Generated Prediction: .
Predicted Sentence: 'Pogoda dzisiaj jest beznadziejna.' Zdanie to ma wydźwięk .

Input Sentence: 'Ten film to był kiler. Nie mogłem się oderwać od ekranu.' Wypowiedź ta ma jest zdecydowanie <mask>
Generated Prediction: .
Predicted Sentence: 'Ten film to był kiler. Nie mogłem się oderwać od ekranu.' Wypowiedź ta ma jest zdecydowanie .



Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForMaskedLM: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing XLMRobertaForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Testing xlm-roberta-base:

Input Sentence: 'To jest mój ulubiony serial!' Zdanie to ma wydźwięk <mask>.
Generated Prediction: .
Predicted Sentence: 'To jest mój ulubiony serial!' Zdanie to ma wydźwięk ..

Input Sentence: 'Jak ładnie dzisiaj jest na dworze!' Zdanie to ma wydźwięk <mask>.
Generated Prediction: nąć
Predicted Sentence: 'Jak ładnie dzisiaj jest na dworze!' Zdanie to ma wydźwięk nąć.

Input Sentence: 'Zycie jest takie piękne!' Zdanie to ma wydźwięk <mask>.
Generated Prediction: nąć
Predicted Sentence: 'Zycie jest takie piękne!' Zdanie to ma wydźwięk nąć.



In [59]:
for s in sentiments2:
    test_cases1(xlm, s)

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForMaskedLM: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing XLMRobertaForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Testing xlm-roberta-base:

Input Sentence: 'Jak ja go nie lubię.' Zdanie to jest nacechowane <mask>.
Generated Prediction: .
Predicted Sentence: 'Jak ja go nie lubię.' Zdanie to jest nacechowane ..

Input Sentence: 'Pogoda dzisiaj jest beznadziejna.' Zdanie to jest nacechowane <mask>.
Generated Prediction: .
Predicted Sentence: 'Pogoda dzisiaj jest beznadziejna.' Zdanie to jest nacechowane ..

Input Sentence: 'Nic mi się nie chce!' Zdanie to jest nacechowane <mask>.
Generated Prediction: .
Predicted Sentence: 'Nic mi się nie chce!' Zdanie to jest nacechowane ..



Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForMaskedLM: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing XLMRobertaForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Testing xlm-roberta-base:

Input Sentence: 'To jest mój ulubiony serial!' Zdanie to jest nacechowane <mask>.
Generated Prediction: .
Predicted Sentence: 'To jest mój ulubiony serial!' Zdanie to jest nacechowane ..

Input Sentence: 'Jak ładnie dzisiaj jest na dworze!' Zdanie to jest nacechowane <mask>.
Generated Prediction: .
Predicted Sentence: 'Jak ładnie dzisiaj jest na dworze!' Zdanie to jest nacechowane ..

Input Sentence: 'Zycie jest takie piękne!' Zdanie to jest nacechowane <mask>.
Generated Prediction: .
Predicted Sentence: 'Zycie jest takie piękne!' Zdanie to jest nacechowane ..



In [60]:
for s in sentiments:
    test_cases1(herbert, s)

Testing allegro/herbert-base-cased:

Input Sentence: 'Jak ja go nie lubię.' Zdanie to ma <mask> wydźwięk
Generated Prediction: swój
Predicted Sentence: 'Jak ja go nie lubię.' Zdanie to ma swój wydźwięk

Input Sentence: 'Pogoda dzisiaj jest beznadziejna.' Zdanie to ma wydźwięk <mask>
Generated Prediction: :
Predicted Sentence: 'Pogoda dzisiaj jest beznadziejna.' Zdanie to ma wydźwięk :

Input Sentence: 'Ten film to był kiler. Nie mogłem się oderwać od ekranu.' Wypowiedź ta ma jest zdecydowanie <mask>
Generated Prediction: .
Predicted Sentence: 'Ten film to był kiler. Nie mogłem się oderwać od ekranu.' Wypowiedź ta ma jest zdecydowanie .

Testing allegro/herbert-base-cased:

Input Sentence: 'To jest mój ulubiony serial!' Zdanie to ma wydźwięk <mask>.
Generated Prediction: polityczny
Predicted Sentence: 'To jest mój ulubiony serial!' Zdanie to ma wydźwięk polityczny.

Input Sentence: 'Jak ładnie dzisiaj jest na dworze!' Zdanie to ma wydźwięk <mask>.
Generated Prediction: symboliczny
Predi

In [61]:
for s in sentiments2:
    test_cases1(herbert, s)

Testing allegro/herbert-base-cased:

Input Sentence: 'Jak ja go nie lubię.' Zdanie to jest nacechowane <mask>.
Generated Prediction: negatywnie
Predicted Sentence: 'Jak ja go nie lubię.' Zdanie to jest nacechowane negatywnie.

Input Sentence: 'Pogoda dzisiaj jest beznadziejna.' Zdanie to jest nacechowane <mask>.
Generated Prediction: smutkiem
Predicted Sentence: 'Pogoda dzisiaj jest beznadziejna.' Zdanie to jest nacechowane smutkiem.

Input Sentence: 'Nic mi się nie chce!' Zdanie to jest nacechowane <mask>.
Generated Prediction: smutkiem
Predicted Sentence: 'Nic mi się nie chce!' Zdanie to jest nacechowane smutkiem.

Testing allegro/herbert-base-cased:

Input Sentence: 'To jest mój ulubiony serial!' Zdanie to jest nacechowane <mask>.
Generated Prediction: emocjami
Predicted Sentence: 'To jest mój ulubiony serial!' Zdanie to jest nacechowane emocjami.

Input Sentence: 'Jak ładnie dzisiaj jest na dworze!' Zdanie to jest nacechowane <mask>.
Generated Prediction: optymizmem
Predicted Sente

In [62]:
sentiment_negative2_papuga = ["'Jak ja go nie lubię.' Zdanie to jest nacechowane", "'Pogoda dzisiaj jest beznadziejna.' Zdanie to jest nacechowane", "'Nic mi się nie chce!' Zdanie to jest nacechowana"]
sentiment_positive2_papuga = ["'To jest mój ulubiony serial!' Zdanie to jest nacechowane", "'Jak ładnie dzisiaj jest na dworze!' Zdanie to jest nacechowane", "'Zycie jest piękne!' Zdanie to jest nacechowane"]

In [63]:
test_cases_papuga(sentiment_negative2_papuga)

Output:----------------------------------------------------------------------------------------------------
Input sentence: 'Jak ja go nie lubię.' Zdanie to jest nacechowane 
Generated prediction: 'Jak ja go nie lubię.' Zdanie to jest nacechowane ironią, ale nie jest to też zdanie, które można by uznać za obraźliwe.
- Nie, nie, nie. Nie, nie. Nie, nie.
Output:----------------------------------------------------------------------------------------------------
Input sentence: 'Pogoda dzisiaj jest beznadziejna.' Zdanie to jest nacechowane 
Generated prediction: 'Pogoda dzisiaj jest beznadziejna.' Zdanie to jest nacechowane negatywnie.'
''W tym roku nie ma śniegu, a w tym roku jest bardzo zimno.' - powiedział.' - 'Nie ma śniegu, jest zimno
Output:----------------------------------------------------------------------------------------------------
Input sentence: 'Nic mi się nie chce!' Zdanie to jest nacechowana 
Generated prediction: 'Nic mi się nie chce!' Zdanie to jest nacechowana ironią,

In [64]:
test_cases_papuga(sentiment_positive2_papuga)

Output:----------------------------------------------------------------------------------------------------
Input sentence: 'To jest mój ulubiony serial!' Zdanie to jest nacechowane 
Generated prediction: 'To jest mój ulubiony serial!' Zdanie to jest nacechowane bardzo emocjonalnie, ale nie jest to też serial, który ogląda się z przyjemnością.
'To jest mój ulubiony serial!' Zdanie to jest nacechowane bardzo
Output:----------------------------------------------------------------------------------------------------
Input sentence: 'Jak ładnie dzisiaj jest na dworze!' Zdanie to jest nacechowane 
Generated prediction: 'Jak ładnie dzisiaj jest na dworze!' Zdanie to jest nacechowane bardzo optymistycznie, a zarazem jest to zdanie, które może być bardzo przydatne w życiu.
'Jak pięknie jest na dworze!' Zdanie to jest nace
Output:----------------------------------------------------------------------------------------------------
Input sentence: 'Zycie jest piękne!' Zdanie to jest nacechowane 
G